# Motor de Decisão - Mercado Físico de Grãos

## Sistema de Suporte à Decisão para Trading de Commodities

Este notebook demonstra o funcionamento do Motor de Decisão, um sistema que transforma sinais dispersos de mercado em recomendações consistentes e auditáveis para operações no mercado físico de soja.

### Arquitetura do Sistema

```
┌─────────────────────────────────────────────────────────────────┐
│                        MARKET INPUTS                            │
│  Premium │ Lineup │ Competitividade │ Câmbio │ Demanda │ Chicago│
└────────────────────────────┬────────────────────────────────────┘
                             │
                             ▼
┌─────────────────────────────────────────────────────────────────┐
│                     SCORING ENGINE                              │
│  Normalização → Ponderação → Score Agregado [0-100]            │
└────────────────────────────┬────────────────────────────────────┘
                             │
                             ▼
┌─────────────────────────────────────────────────────────────────┐
│                    OVERRIDE RULES                               │
│  Logística │ Queda Conjunta │ Armadilha │ Competitividade       │
└────────────────────────────┬────────────────────────────────────┘
                             │
                             ▼
┌─────────────────────────────────────────────────────────────────┐
│                   BOOK MODULATION                               │
│  Limites de Exposição │ Meta de Hedge │ Sizing Efetivo         │
└────────────────────────────┬────────────────────────────────────┘
                             │
                             ▼
┌─────────────────────────────────────────────────────────────────┐
│                      DECISION OUTPUT                            │
│  Recomendação Física │ Recomendação Hedge │ Justificativa       │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Setup e imports
import sys
from pathlib import Path
from datetime import date, timedelta

# Adicionar paths
ROOT = Path().absolute().parent
sys.path.insert(0, str(ROOT / "src"))
sys.path.insert(0, str(ROOT / "data"))

# Imports do motor
from engine import DecisionEngine, MarketInputs, DecisionReport
from book import BookState
from scoring import compute_scoring, PhysicalClassification
from overrides import OverrideType
from mock_generator import (
    generate_scenario_normal,
    generate_scenario_crisis,
    generate_3y_history,
    MarketDataGenerator,
)

# Visualizacao
try:
    import matplotlib.pyplot as plt
    import matplotlib.dates as mdates
    PLOT_AVAILABLE = True
    plt.style.use('seaborn-v0_8-whitegrid')
    plt.rcParams['figure.figsize'] = (12, 6)
    plt.rcParams['font.size'] = 10
except ImportError:
    PLOT_AVAILABLE = False
    print("matplotlib nao disponivel - graficos desabilitados")

print("Setup completo!")

## 1. Dados de Mercado Simulados

O gerador de dados cria séries temporais realistas com:
- **Sazonalidade**: Safra (Mar-Jul) vs Entressafra
- **Correlações**: Entre variáveis de mercado
- **Eventos**: Spikes, crises, volatilidade

In [ ]:
# Gerar 3 anos de historico
print("Gerando 3 anos de dados historicos...")
history = generate_3y_history(seed=42)
print(f"Gerados {len(history)} dias de dados")
print(f"Periodo: {history[0].date} a {history[-1].date}")

# Estatisticas
premiums = [d.premium_paranagua for d in history]
lineups = [d.lineup_bruto for d in history]
chicagos = [d.chicago_front for d in history]

print(f"\nEstatisticas:")
print(f"  Premium: min={min(premiums):.0f}, max={max(premiums):.0f}, media={sum(premiums)/len(premiums):.0f} c/bu")
print(f"  Lineup:  min={min(lineups)}, max={max(lineups)}, media={sum(lineups)/len(lineups):.0f} navios")
print(f"  Chicago: min={min(chicagos):.0f}, max={max(chicagos):.0f}, media={sum(chicagos)/len(chicagos):.0f} c/bu")

In [ ]:
# Visualizacao dos dados
if PLOT_AVAILABLE:
    fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)
    
    dates = [d.date for d in history]
    
    # Premium
    ax1 = axes[0]
    ax1.plot(dates, premiums, color='#2E86AB', linewidth=1)
    ax1.fill_between(dates, premiums, alpha=0.3, color='#2E86AB')
    ax1.set_ylabel('Premium (¢/bu)')
    ax1.set_title('Premio FOB Paranagua', fontweight='bold')
    ax1.axhline(y=80, color='gray', linestyle='--', alpha=0.5, label='Base')
    
    # Lineup
    ax2 = axes[1]
    ax2.bar(dates, lineups, color='#A23B72', alpha=0.7, width=1)
    ax2.set_ylabel('Navios')
    ax2.set_title('Line-up Bruto', fontweight='bold')
    
    # Chicago
    ax3 = axes[2]
    ax3.plot(dates, chicagos, color='#F18F01', linewidth=1)
    ax3.fill_between(dates, chicagos, alpha=0.3, color='#F18F01')
    ax3.set_ylabel('Chicago (¢/bu)')
    ax3.set_title('CBOT Soja Front Month', fontweight='bold')
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%b/%y'))
    ax3.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    
    plt.tight_layout()
    plt.suptitle('Dados de Mercado - 3 Anos de Historico', fontsize=14, fontweight='bold', y=1.02)
    plt.show()
else:
    print("Graficos nao disponiveis sem matplotlib")

## 2. Motor de Decisão em Ação

O motor combina múltiplos indicadores em um score agregado [0-100]:

| Componente | Peso | Descrição |
|------------|------|------------|
| Line-up | 30% | Demanda real (navios programados) |
| Prêmio | 25% | Nível de preço vs histórico |
| Competitividade | 20% | Brasil vs US Gulf |
| Demanda | 15% | Pace de exportação |
| Câmbio | 10% | Modulador de margem |

In [ ]:
# Cenario 1: Mercado Normal
print("=" * 60)
print("CENARIO 1: Mercado Normal")
print("=" * 60)

inputs_normal = MarketInputs(
    dt=date(2024, 5, 15),
    var_semanal_lineup=5.0,      # Leve alta
    percentil_premium=65.0,       # Acima da media
    spread_adjusted=-5.0,         # Brasil competitivo
    z_pace=0.3,                   # Demanda normal
    var_cambio_5d=-0.5,           # Real levemente forte
    chicago_percentile=55.0,
    chicago_is_spike=False,
    logistics_flag_active=False,
    logistics_reason=None,
)

engine = DecisionEngine()
report = engine.run(inputs_normal)

print(f"\nScore Fisico: {report.score_fisico:.1f}/100 ({report.classificacao})")
print(f"\nComponentes:")
for name, comp in report.componentes.items():
    bar = '█' * int(comp['score'] / 5) + '░' * (20 - int(comp['score'] / 5))
    print(f"  {name:15} [{bar}] {comp['score']:.0f}")

print(f"\nRecomendacao Fisica: {report.recomendacao_fisica['acao'].upper()}")
print(f"Recomendacao Hedge:  {report.recomendacao_hedge['acao'].upper()}")

In [ ]:
# Cenario 2: Override Ativado - Armadilha de Premio
print("=" * 60)
print("CENARIO 2: Override - Armadilha de Premio")
print("=" * 60)
print("\nSituacao: Premio alto (85%) MAS lineup caindo (-12%)")
print("Interpretacao: Preco ainda nao corrigiu, demanda ja saiu\n")

inputs_trap = MarketInputs(
    dt=date(2024, 4, 20),
    var_semanal_lineup=-12.0,     # CAINDO
    percentil_premium=85.0,        # ALTO - armadilha!
    spread_adjusted=-8.0,
    z_pace=0.5,
    var_cambio_5d=0.2,
    chicago_percentile=60.0,
    chicago_is_spike=False,
    logistics_flag_active=False,
    logistics_reason=None,
)

report_trap = engine.run(inputs_trap)

print(f"Score Fisico: {report_trap.score_fisico:.1f}/100")
print(f"\n⚠️  OVERRIDE ATIVO: {report_trap.override_dominante}")
print(f"\nRecomendacao Original (sem override): provavelmente AUMENTAR")
print(f"Recomendacao Final:    {report_trap.recomendacao_fisica['acao'].upper()}")
print(f"Sizing:                {report_trap.recomendacao_fisica['sizing_pct']:+.0f}%")
print(f"\nJustificativa: {report_trap.justificativa}")

In [ ]:
# Cenario 3: Modulacao por Book
print("=" * 60)
print("CENARIO 3: Modulacao por Limite de Book")
print("=" * 60)
print("\nSituacao: Mercado muito forte, mas ja estamos no limite\n")

inputs_strong = MarketInputs(
    dt=date(2024, 6, 5),
    var_semanal_lineup=15.0,      # Muito forte
    percentil_premium=82.0,        # Alto
    spread_adjusted=-18.0,         # Brasil muito barato
    z_pace=1.5,                    # Demanda forte
    var_cambio_5d=-2.0,            # Real forte
    chicago_percentile=70.0,
    chicago_is_spike=False,
    logistics_flag_active=False,
    logistics_reason=None,
)

# Book no limite
book_limite = BookState(
    exposicao_fisica_pct=80.0,    # NO LIMITE
    limite_long_pct=80.0,
    limite_short_pct=-50.0,
    hedge_atual_pct=60.0,
    hedge_meta_pct=60.0,
)

engine_book = DecisionEngine(book_limite)
report_book = engine_book.run(inputs_strong)

print(f"Score Fisico: {report_book.score_fisico:.1f}/100 (MUITO FORTE)")
print(f"\nEstado do Book:")
print(f"  Exposicao atual: {book_limite.exposicao_fisica_pct}%")
print(f"  Limite long:     {book_limite.limite_long_pct}%")
print(f"\n📊 MODULACAO APLICADA")
print(f"Recomendacao Original: AUMENTAR_FORTE (+25%)")
print(f"Recomendacao Final:    {report_book.recomendacao_fisica['acao'].upper()}")
print(f"Razao: {report_book.modulacao_razao}")

## 3. Hierarquia de Overrides

Os overrides são regras que **dominam** o scoring quando ativadas, representando situações de mercado que raramente falham.

```
Prioridade (maior → menor):

1. LOGÍSTICA          → Greve, congestionamento portuário
2. QUEDA_CONJUNTA     → Lineup caindo E prêmio caindo
3. ARMADILHA_PREMIO   → Prêmio alto E lineup caindo
4. COMPETITIVIDADE    → Spread > +15 USD/ton
5. CHICAGO_ESPECULATIVO → Spike sem fundamento
```

In [ ]:
# Demonstracao de todos os overrides
overrides_demo = [
    ("LOGISTICA", MarketInputs(
        dt=date(2024, 3, 15),
        var_semanal_lineup=5.0, percentil_premium=70.0, spread_adjusted=-5.0,
        z_pace=0.5, var_cambio_5d=0.0, chicago_percentile=50.0,
        chicago_is_spike=False, logistics_flag_active=True,
        logistics_reason="Greve de caminhoneiros"
    )),
    ("QUEDA_CONJUNTA", MarketInputs(
        dt=date(2024, 4, 10),
        var_semanal_lineup=-15.0, percentil_premium=25.0, spread_adjusted=5.0,
        z_pace=-0.8, var_cambio_5d=1.5, chicago_percentile=40.0,
        chicago_is_spike=False, logistics_flag_active=False, logistics_reason=None
    )),
    ("ARMADILHA_PREMIO", MarketInputs(
        dt=date(2024, 5, 5),
        var_semanal_lineup=-12.0, percentil_premium=88.0, spread_adjusted=-3.0,
        z_pace=0.3, var_cambio_5d=-0.5, chicago_percentile=55.0,
        chicago_is_spike=False, logistics_flag_active=False, logistics_reason=None
    )),
    ("COMPETITIVIDADE", MarketInputs(
        dt=date(2024, 8, 20),
        var_semanal_lineup=3.0, percentil_premium=50.0, spread_adjusted=22.0,
        z_pace=0.0, var_cambio_5d=0.5, chicago_percentile=45.0,
        chicago_is_spike=False, logistics_flag_active=False, logistics_reason=None
    )),
    ("CHICAGO_ESPECULATIVO", MarketInputs(
        dt=date(2024, 7, 1),
        var_semanal_lineup=2.0, percentil_premium=55.0, spread_adjusted=0.0,
        z_pace=0.2, var_cambio_5d=0.0, chicago_percentile=78.0,
        chicago_is_spike=True, logistics_flag_active=False, logistics_reason=None
    )),
]

print("DEMONSTRACAO DE OVERRIDES")
print("=" * 70)
print(f"{'Override':<25} {'Score':>8} {'Acao Fisica':<18} {'Sizing':>8}")
print("-" * 70)

engine = DecisionEngine()
for name, inputs in overrides_demo:
    report = engine.run(inputs)
    print(f"{name:<25} {report.score_fisico:>8.1f} {report.recomendacao_fisica['acao']:<18} {report.recomendacao_fisica['sizing_pct']:>+7.0f}%")

## 4. Output JSON Estruturado

Cada execução do motor gera um relatório JSON completo, auditável e integrável com outros sistemas.

In [ ]:
# Exemplo de output completo
import json

inputs_example = MarketInputs(
    dt=date(2024, 5, 20),
    var_semanal_lineup=8.0,
    percentil_premium=72.0,
    spread_adjusted=5.0,
    z_pace=0.5,
    var_cambio_5d=-0.8,
    chicago_percentile=65.0,
    chicago_is_spike=False,
    logistics_flag_active=False,
    logistics_reason=None,
)

report = engine.run(inputs_example)
print(report.to_json(indent=2))

## 5. Análise de Sensibilidade

Como o score varia com mudanças nos inputs?

In [ ]:
# Sensibilidade ao lineup
lineup_range = range(-20, 21, 5)
scores_by_lineup = []

for var_lineup in lineup_range:
    inputs = MarketInputs(
        dt=date(2024, 5, 15),
        var_semanal_lineup=float(var_lineup),
        percentil_premium=50.0,  # Fixo em neutro
        spread_adjusted=0.0,
        z_pace=0.0,
        var_cambio_5d=0.0,
        chicago_percentile=50.0,
        chicago_is_spike=False,
        logistics_flag_active=False,
        logistics_reason=None,
    )
    report = engine.run(inputs)
    scores_by_lineup.append((var_lineup, report.score_fisico))

print("Sensibilidade: Variacao Semanal do Lineup vs Score Fisico")
print("=" * 50)
for var, score in scores_by_lineup:
    bar = '█' * int(score / 5)
    print(f"  Lineup {var:+3.0f}% → Score {score:5.1f} [{bar}]")

In [ ]:
# Visualizacao de sensibilidade
if PLOT_AVAILABLE:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Lineup
    ax1 = axes[0]
    vars_l, scores_l = zip(*scores_by_lineup)
    ax1.plot(vars_l, scores_l, 'o-', color='#2E86AB', linewidth=2, markersize=8)
    ax1.axhline(y=50, color='gray', linestyle='--', alpha=0.5)
    ax1.axhline(y=65, color='green', linestyle='--', alpha=0.5, label='Forte')
    ax1.axhline(y=35, color='red', linestyle='--', alpha=0.5, label='Fraco')
    ax1.set_xlabel('Variacao Semanal Lineup (%)')
    ax1.set_ylabel('Score Fisico')
    ax1.set_title('Sensibilidade: Lineup', fontweight='bold')
    ax1.legend()
    ax1.set_ylim(0, 100)
    
    # Premium
    premium_range = range(0, 101, 10)
    scores_by_premium = []
    for pct in premium_range:
        inputs = MarketInputs(
            dt=date(2024, 5, 15),
            var_semanal_lineup=0.0,
            percentil_premium=float(pct),
            spread_adjusted=0.0, z_pace=0.0, var_cambio_5d=0.0,
            chicago_percentile=50.0, chicago_is_spike=False,
            logistics_flag_active=False, logistics_reason=None,
        )
        report = engine.run(inputs)
        scores_by_premium.append((pct, report.score_fisico))
    
    ax2 = axes[1]
    pcts, scores_p = zip(*scores_by_premium)
    ax2.plot(pcts, scores_p, 's-', color='#A23B72', linewidth=2, markersize=8)
    ax2.axhline(y=50, color='gray', linestyle='--', alpha=0.5)
    ax2.axhline(y=65, color='green', linestyle='--', alpha=0.5)
    ax2.axhline(y=35, color='red', linestyle='--', alpha=0.5)
    ax2.set_xlabel('Percentil Premio')
    ax2.set_ylabel('Score Fisico')
    ax2.set_title('Sensibilidade: Premio', fontweight='bold')
    ax2.set_ylim(0, 100)
    
    plt.tight_layout()
    plt.show()

## Conclusão

O Motor de Decisão demonstra:

1. **Agregação inteligente** de múltiplos sinais de mercado
2. **Regras de override** que capturam situações críticas
3. **Modulação por posição** para gestão de risco
4. **Rastreabilidade** completa das decisões

### Próximos Passos

- [ ] Backtest com dados históricos reais
- [ ] Calibração de pesos via otimização
- [ ] Dashboard interativo
- [ ] Integração com sistemas de trading